In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!nvidia-smi

Sat Mar 14 12:11:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
import os 

os.chdir('drive/My Drive/')

FileNotFoundError: ignored

In [ ]:
import pandas as pd
df = pd.read_csv("nCov_10k_test.csv")
df.head(5)

,微博id,微博发布时间,发布人账号,微博中文内容,微博图片,微博视频
0,4456068992182160,01月01日 23:38,-精緻的豬豬女戰士-,#你好2020#新年第一天元气满满的早起出门买早饭结果高估了自己抗冻能力回家成功冻发烧（大概...,['https://ww2.sinaimg.cn/thumb150/745aa591ly1g...,[]
1,4456424178427250,01月02日 23:09,liujunyi88,大宝又感冒鼻塞咳嗽了，还有发烧。队友加班几天不回。感觉自己的情绪在家已然是随时引爆的状态。情...,[],[]
2,4456797466940200,01月03日 23:53,ablsa,还要去输两天液，这天也太容易感冒发烧了，一定要多喝热水啊?,['https://ww3.sinaimg.cn/orj360/006fTidCly1gaj...,[]
3,4456791021108920,01月03日 23:27,喵吃鱼干Lynn,我太难了别人怎么发烧都没事就我一检查甲型流感?,[],[]
4,4457086404997440,01月04日 19:01,我的发小今年必脱单,果然是要病一场的喽回来第三天开始感冒今儿还发烧了喉咙眼睛都难受的一匹怎么样能不经意让我的毕设...,[],[]


In [ ]:
import jieba
import pandas as pd
import numpy as np
import warnings
from gensim.models.word2vec import Word2Vec
from gensim.models.word2vec import PathLineSentences
import gensim
from itertools import chain
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
import torch.autograd as autograd
import torchtext.vocab as torchvocab
from torch.autograd import Variable
from sklearn.metrics import accuracy_score,recall_score,f1_score
import time

In [ ]:
train_df = pd.read_csv('nCoV_100k_train.labled.csv',)
train_df['word']=train_df["微博中文内容"]
train_df['label']=train_df["情感倾向"]
train_df['wordcut']=train_df['word'].astype(str).apply(lambda x: jieba.lcut(x))
train_df1 = train_df.copy()
train_df1=train_df1.drop(labels=['微博id','微博发布时间',"发布人账号",'微博中文内容','微博图片','微博视频','情感倾向'],axis=1)
letters = ['word','wordcut','label']
train_df1=train_df1[letters]
train_df1=train_df1[~train_df1['label'].isin(['9','-','·'])]
train_df1 = train_df1.fillna(10)
train_df1=train_df1[~train_df1['label'].isin(['-2','10','4',10])]
train_df1.info()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.649 seconds.
Prefix dict has been built successfully.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 99913 entries, 0 to 99999
Data columns (total 3 columns):
word       99913 non-null object
wordcut    99913 non-null object
label      99913 non-null object
dtypes: object(3)
memory usage: 3.0+ MB


In [ ]:
test_df = pd.read_csv('nCov_10k_test.csv',header= 0)
test_df['word']=test_df["微博中文内容"]
test_df['label']=None
test_df['wordcut']=test_df['word'].astype(str).apply(lambda x: jieba.lcut(x))
df1_test = test_df.copy()
df1_test=df1_test.drop(labels=['微博id','微博发布时间',"发布人账号",'微博中文内容','微博图片','微博视频'],axis=1)
letters_test = ['word','wordcut','label']
df1_test=df1_test[letters_test]
test_tokenized = df1_test['wordcut'].tolist()
len(test_tokenized)

10000

In [ ]:
w2v = Word2Vec(size=100,min_count=10)
x = train_df1['wordcut']
w2v.build_vocab(x)
w2v.train(x, total_examples=w2v.corpus_count,epochs=w2v.epochs)
w2v.save("Word2vec_1003k.w2v")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
train_df1['label'].loc[train_df1['label']== '-1'] = '2'

In [ ]:
wvmodelwvmodel = gensim.models.word2vec.Word2Vec.load("Word2vec_1003k.w2v").wv
vocab = set(chain(*train_df1['wordcut'].tolist()))
vocab_size = len(vocab)
train_tokenized = train_df1['wordcut'].tolist()
word_to_idx = {word: i+1 for i, word in enumerate(vocab)}
word_to_idx['<unk>'] = 0
idx_to_word = {i+1: word for i, word in enumerate(vocab)}
idx_to_word[0] = '<unk>'

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
def encode_samples(tokenized_samples,vocab):
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            if token in word_to_idx:
                feature.append(word_to_idx[token])
            else:
                feature.append(0)
        features.append(feature)
    return features
def pad_samples(features,maxlen=100,PAD=0):
    padded_features = []
    for feature in features:
        if len(feature) >=maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            while(len(padded_feature) <maxlen):
                padded_feature.append(PAD)
        padded_features.append(padded_feature)
    return padded_features

In [ ]:
train_features = torch.tensor(pad_samples(encode_samples(train_tokenized,vocab)))
train_labels = torch.tensor([int(score) for score in train_df1["label"].tolist()])


In [ ]:
class SentimentNet(nn.Module):
    def __init__(self,vocab_size,embed_size,num_hiddens,num_layers,
                bidirectional,weight,labels,use_gpu,**kwargs):
        super(SentimentNet,self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        
        self.bidirectional = bidirectional
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad =False
        self.encoder = nn.LSTM(input_size=embed_size, hidden_size=self.num_hiddens,
                               num_layers=num_layers, bidirectional=self.bidirectional,
                               dropout=0)
        if self.bidirectional:
            self.decoder = nn.Linear(num_hiddens * 4,labels)
        else:
            self.decoder = nn.Linear(num_hiddens * 2,labels)
            
    def forward(self,inputs):
        embeddings = self.embedding(inputs)
        states,hidden = self.encoder(embeddings.permute([1,0,2]))
        encoding = torch.cat([states[0],states[-1]],dim=1)
        outputs = self.decoder(encoding)
        return outputs


In [ ]:
num_epochs = 3
embed_size = 100
num_hiddens = 100
num_layers = 2
bidirectional = True
batch_size = 64
labels = 3
lr = 0.01
device = torch.device('cuda:0')
use_gpu = True

In [ ]:
weight = torch.zeros(vocab_size+1, embed_size)
for i in range(len(wvmodelwvmodel.index2word)):
    try:
        index = word_to_idx[wvmodelwvmodel.index2word[i]]
    except:
        continue
    weight[index,:] = torch.from_numpy(wvmodelwvmodel.get_vector(
    idx_to_word[word_to_idx[wvmodelwvmodel.index2word[i]]]))

net = SentimentNet(vocab_size=(vocab_size+1), embed_size=embed_size,
                   num_hiddens=num_hiddens, num_layers=num_layers,
                   bidirectional=bidirectional, weight=weight,
                   labels=labels, use_gpu=use_gpu)
net.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

In [ ]:
print("data load...")
train_set = torch.utils.data.TensorDataset(train_features, train_labels)
train_iter = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=True)
test_features = torch.tensor(pad_samples(encode_samples(test_tokenized, vocab)))
test_labels = torch.tensor([0 for score in range(10000)])

test_set = torch.utils.data.TensorDataset(test_features, test_labels)
test_iter = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                        shuffle=False)

data load...


In [ ]:
len(train_labels)

99913

In [ ]:

print("model train ...")
predlist = []

for epoch in range(num_epochs):
    start = time.time()
    train_loss = 0
    train_acc = 0
    train_rec = 0
    train_f1 = 0
    n= 0
    for feature,label in train_iter:
        n += 1
        net.zero_grad()
        feature = Variable(feature.cuda())
        label = Variable(label.cuda())
        score = net(feature)
        loss = loss_function(score, label)
        loss.backward()
        optimizer.step()
        train_acc += accuracy_score(torch.argmax(score.cpu().data,dim=1),label.cpu())
        train_rec += recall_score(torch.argmax(score.cpu().data,dim=1),label.cpu(),average='macro')
        train_f1 += f1_score(torch.argmax(score.cpu().data,dim=1),label.cpu(),average='macro')

        train_loss += loss
    
    with torch.no_grad():
        for test_feature, test_label in test_iter:
            test_feature = Variable(test_feature.cuda())
            test_label = Variable(test_label)
            test_score = net(test_feature)
            predlist.append(torch.argmax(test_score.data,dim=1))          
    # print(predlist)
    if epoch < num_epochs-1:
        predlist = []
    print(predlist)
    end = time.time()
    runtime = end-start
    print('epoch: %d, train loss: %.4f,train acc: %.2f,train recall: %.2f,train f1: %.2f,time: %.2f' %
         (epoch,train_loss.data / n, train_acc / n , train_rec / n , train_f1 / n , runtime))
print("model train end")
torch.save (net.state_dict(), 'params3.pk1') 


epoch: 0, train loss: 0.6857,train acc: 0.70,train recall: 0.67,train f1: 0.62,time: 31.50


epoch: 1, train loss: 0.6262,train acc: 0.73,train recall: 0.71,train f1: 0.66,time: 31.53

epoch: 2, train loss: 0.6115,train acc: 0.73,train recall: 0.71,train f1: 0.67,time: 31.87


model train end

In [ ]:
len(predlist)

157

In [ ]:
net.load_state_dict(torch.load('params3.pk1'))  #加载模型
predlist1 = []

for test_feature, test_label in test_iter:
    test_feature = Variable(test_feature.cuda())
    test_label = Variable(test_label.cuda())
    test_score = net(test_feature)
    predlist1.append(torch.argmax(test_score.data,dim=1))          
print(predlist1)

In [ ]:
0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0
0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0
len(predlist1)

157

In [ ]:
abblist=[]
for i in range(157):
    if i<156:
        for j in range(64): 
            if int(predlist[i][j]) ==2:
                abblist.append(-1)
            else:
                abblist.append(int(predlist[i][j]))
    else:
        for j in range(16):
            if int(predlist[i][j]) ==2:
                abblist.append(-1)
            else:
                abblist.append(int(predlist[i][j]))
dic={}
for i in abblist:
    dic[i] = abblist.count(i)
dic

{-1: 1298, 0: 6560, 1: 2142}

In [ ]:
abblist1=[]
for i in range(157):
    if i<156:
        for j in range(64): 
            if int(predlist1[i][j]) ==2:
                abblist1.append(-1)
            else:
                abblist1.append(int(predlist[i][j]))
    else:
        for j in range(16):
            if int(predlist1[i][j]) ==2:
                abblist1.append(-1)
            else:
                abblist1.append(int(predlist[i][j]))
dic1={}
for i in abblist1:
    dic1[i] = abblist1.count(i)
dic1

{-1: 1298, 0: 6560, 1: 2142}